In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import torch


### Webcam object Detection by using VGG16 pretrained weights that only detect one object in each frame

In [ ]:
model = VGG16(weights='imagenet')

In [ ]:
def inference(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    return decode_predictions(preds, top=1)[0][0][1]

In [ ]:
import cv2
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (224, 224))
    predicted = inference(frame[...,::-1])
    cv2.putText(frame,  predicted, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), lineType=cv2.LINE_AA)
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): #q is the Enter Key
        break
cap.release()
cv2.destroyAllWindows() 

### Webcam object Detection which is able to detect multiple objects in each frame using YOLO5 pretrained weights

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
def draw_bounding_boxes(image, boxes, idxs):
    if idxs > 0:
        for i in range(idxs):
            # extract bounding box coordinates
            xmin, ymin = int(boxes.iloc[i][0]),  int(boxes.iloc[i][1])
            xmax, ymax =  int(boxes.iloc[i][2]),  int(boxes.iloc[i][3])

            # draw the bounding box and label on the image
            #color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255,0,0), 2)
            text = "{}: {:.4f}".format( boxes.iloc[i][6],  boxes.iloc[i][4])
            cv2.putText(image, text, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,0,0), 2)

    return image

In [ ]:
import cv2
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    results=model(frame)
    image=draw_bounding_boxes(frame, results.pandas().xyxy[0], len(results.pandas().xyxy[0]))
    cv2.imshow('Webcam', image)
    if cv2.waitKey(1) & 0xFF == ord('q'): #q is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()  